In [1]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.autograd import Variable
import torch
from tqdm import tqdm
import sys

In [2]:
sys.path.insert(0,'..')
import bitmap
import itertools
from forward_prediction import forward_model

# Generate data

## Training data

In [3]:
# Generate training data
N = 25600
train_data_gen = bitmap.generate_train_set(N, 41, min_delta=1, max_delta=1)
deltas, start_boards, stop_boards = map(np.array, zip(*list(train_data_gen)))

## Test data

In [4]:
# Generate testing data
N_test = 25600
test_data_gen = bitmap.generate_train_set(N, 42, min_delta=1, max_delta=1)
deltas, test_start_boards, test_stop_boards = map(np.array, zip(*list(test_data_gen)))

# Model trainer

In [37]:
def train(model, X, y, optim, criterion, output_path, num_epochs=50, batch_size=128):
    X_ = X.cuda()
    y_ = y.cuda()
    model.cuda()
    print(model)
    # Set optimizer
    optimizer = optim(model.parameters())
    for epoch in tqdm(range(num_epochs)): 
        permutation = torch.randperm(X_.size()[0])
        running_loss = 0.0
        for i in range(0, X_.size()[0], batch_size):
            indices = permutation[i:i+batch_size]
            batch = X_[indices]
            target = y_[indices]
        
            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 4000 == 0 and i > 0:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
    # Save model state_dict
    torch.save(model.state_dict(), output_path)

# Train a forward network

In [28]:
class ForwardNet(nn.Module):
    def __init__(self):
        super(ForwardNet, self).__init__()
        # in channels, out channels, kernel size
        self.conv1 = nn.Conv2d(1, 16, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 8, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ2 = nn.PReLU()
        self.conv3 = nn.Conv2d(8, 4, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ3 = nn.PReLU()
        self.conv4 = nn.Conv2d(4, 1, (3, 3), padding=(1, 1), padding_mode='circular')
        
    def forward(self, x):
        x = self.activ1(self.conv1(x))
        x = self.activ2(self.conv2(x))
        x = self.activ3(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [29]:
forward_net = ForwardNet()
criterion = nn.BCELoss()

In [31]:
X = Variable(torch.tensor(start_boards).view(N, 1, 25, 25).float(), requires_grad=True)
y = Variable(torch.tensor(stop_boards).view(N, 1, 25, 25).float())

In [32]:
train(forward_net, X, y, optim.Adam, criterion, "../models/johnson/vanilla_forward.pkl")

  0%|          | 0/50 [00:00<?, ?it/s]

ForwardNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ1): ReLU()
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ2): PReLU(num_parameters=1)
  (conv3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ3): PReLU(num_parameters=1)
  (conv4): Conv2d(4, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
)
[1, 16001] loss: 0.033


  2%|▏         | 1/50 [00:10<08:27, 10.36s/it]

[2, 16001] loss: 0.028


  4%|▍         | 2/50 [00:20<08:15, 10.33s/it]

...

 96%|█████████▌| 48/50 [08:24<00:21, 10.56s/it]

[49, 16001] loss: 0.000


 98%|█████████▊| 49/50 [08:35<00:10, 10.56s/it]

[50, 16001] loss: 0.000


100%|██████████| 50/50 [08:45<00:00, 10.52s/it]


In [33]:
def get_forward_mae(start_boards, stop_boards, n):
    start_boards_tensor = torch.tensor(start_boards).view(n, 1, 25, 25).float().cuda()
    stop_boards_tensor = torch.tensor(stop_boards).view(n, 1, 25, 25)
    with torch.no_grad():
        forward_net.eval()
        # Make prediction
        predicted_stop_board = (forward_net(start_boards_tensor) > 0.5).int().cpu()
        error = torch.sum(predicted_stop_board != stop_boards_tensor)
        # print(predicted_stop_board)
        # print(stop_boards_tensor)
        return error / (n * 25 * 25)

In [34]:
# Training data MAE
train_mae = get_forward_mae(start_boards, stop_boards, N)
print("The training data MAE is {:.6f}.".format(train_mae))

# Test data MAE
test_mae = get_forward_mae(test_start_boards, test_stop_boards, N_test)
print("The test data MAE is {:.6f}.".format(test_mae))

The training data MAE is 0.000000.
The test data MAE is 0.000000.


# Relax starting boards

In [35]:
# Modify starting boards 
def relax_boards(boards):
    return np.abs(np.random.rand(*boards.shape) / 2 - boards)

In [36]:
relaxed_start_boards = relax_boards(start_boards)

In [38]:
X_relaxed = Variable(torch.tensor(relaxed_start_boards).view(N, 1, 25, 25).float(), requires_grad=True)
y = Variable(torch.tensor(stop_boards).view(N, 1, 25, 25).float())

In [ ]:
train(forward_net, X_relaxed, y, optim.Adam, criterion, "../models/johnson/forward_with_relaxation.pkl")

  0%|          | 0/50 [00:00<?, ?it/s]

ForwardNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ1): ReLU()
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ2): PReLU(num_parameters=1)
  (conv3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ3): PReLU(num_parameters=1)
  (conv4): Conv2d(4, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
)
[1, 16001] loss: 0.045


  2%|▏         | 1/50 [00:10<08:39, 10.60s/it]

[2, 16001] loss: 0.032


  4%|▍         | 2/50 [00:21<08:29, 10.61s/it]

...

 36%|███▌      | 18/50 [03:16<05:57, 11.16s/it]

[19, 16001] loss: 0.019


 38%|███▊      | 19/50 [03:27<05:46, 11.18s/it]

[20, 16001] loss: 0.020


 40%|████      | 20/50 [03:39<05:36, 11.23s/it]

# Reverse model

In [4]:
class ReverseNet(nn.Module):
    def __init__(self):
        super(ReverseNet, self).__init__()
        # in channels, out channels, kernel size
        self.conv1 = nn.Conv2d(1, 16, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ1 = nn.PReLU()
        self.conv2 = nn.Conv2d(16, 8, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ2 = nn.PReLU()
        self.conv3 = nn.Conv2d(8, 4, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ3 = nn.PReLU()
        self.conv4 = nn.Conv2d(4, 1, (3, 3), padding=(1, 1), padding_mode='circular')

    def forward(self, x):
        x = self.reverse(x)
        return x

    def reverse(self, x):
        x = self.activ1(self.conv1(x))
        x = self.activ2(self.conv2(x))
        x = self.activ3(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [5]:
net = ReverseNet()
print(net)

ReverseNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=circular)
  (activ1): PReLU(num_parameters=1)
  (conv2): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=circular)
  (activ2): PReLU(num_parameters=1)
  (conv3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ3): PReLU(num_parameters=1)
  (conv4): Conv2d(4, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
)


In [6]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters()) #, lr=0.001, momentum=0.9)

In [10]:
X = Variable(torch.tensor(stop_boards).view(N, 1, 25, 25).float(), requires_grad=True)
y = Variable(torch.tensor(start_boards).view(N, 1, 25, 25).float())
num_epochs = 50
batch_size = 128
for epoch in tqdm(range(num_epochs)): 
    permutation = torch.randperm(X.size()[0])
    running_loss = 0.0
    for i in range(0, X.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch = X[indices]
        target = y[indices]
        
        optimizer.zero_grad()
        outputs = net(batch)
        # loss = criterion(outputs, batch)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 0 and i > 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0

  0%|          | 0/50 [00:00<?, ?it/s]

[1, 16001] loss: 0.037


  2%|▏         | 1/50 [00:17<14:31, 17.78s/it]

[2, 16001] loss: 0.034


  4%|▍         | 2/50 [00:35<14:17, 17.87s/it]

...

 96%|█████████▌| 48/50 [14:33<00:37, 18.74s/it]

[49, 16001] loss: 0.036


 98%|█████████▊| 49/50 [14:53<00:19, 19.16s/it]

[50, 16001] loss: 0.037


100%|██████████| 50/50 [15:14<00:00, 18.28s/it]


In [11]:
# Generate testing data
N_test = 25600
# test_data_gen = bitmap.generate_test_set(N_test, 45, max_delta=1)
test_data_gen = bitmap.generate_test_set(N_test, 45, min_delta=5, max_delta=5)
delta, test_stop_boards = map(list, zip(*list(test_data_gen)))

In [17]:
def get_mae(stop_boards, n, delta=1):
    stop_boards_tensor = torch.tensor(stop_boards).view(n, 1, 25, 25).float()
    with torch.no_grad():
        net.eval()
        # Make prediction
        output = (net(stop_boards_tensor) > 0.5).int()
        # Evolve one step forward
        predicted_stop_board = output
        for _ in range(delta):
            predicted_stop_board = (forward_model.forward(predicted_stop_board) > 0.5).int()
        error = torch.sum(predicted_stop_board != stop_boards_tensor.int())
        return error / (n * 25 * 25)

In [13]:
# Training data MAE
train_mae = get_mae(stop_boards, N, 5).item()
print("The training data MAE is {:.6f}.".format(train_mae))

# Test data MAE
test_mae = get_mae(test_stop_boards, N_test, 5).item()
print("The test data MAE is {:.6f}.".format(test_mae))

The training data MAE is 0.137085.
The test data MAE is 0.136620.


In [18]:
# Test data MAE
test_mae = get_mae(test_stop_boards, N_test, 5).item()
print("The test data MAE is {:.6f}.".format(test_mae))

The test data MAE is 0.140959.
